# Relationship between Zillow Rent Index of CA Cities and Nearby Venues

## 0.1 Introduction/Business Problem
### Tenants are looking for an affordable city or area to live and work.The nearby venues are an important factor to consider.
### Investors search a good place to buy houses and use them to collect rentals. The nearby venues can predict the capital return and potential rentals.
### Therefore, this project will explore the relationship between Zillow Rent Index of CA cities and nearby venues from Foursquare.
### This project helps a group of stakeholders (tenants and landlord) solve a problem.

## 0.2 Data Source
### 0.2.1 Zillow Rent Index data from Zillow
### Zillow Rent Index (ZRI): A smoothed measure of the typical estimated market rate rent across a given region and housing type. ZRI, which is a dollar-denominated alternative to repeat-rent indices, is the mean of rent estimates that fall into the 40th to 60th percentile range for all homes and apartments in a given region, including those not currently listed for rent.
### Multifamily, SFR, Condo, Co-op (rental house type)
### City level (California)
### Columns: City Area Size (SizeRank), Rentals (Zri), Each City Rental Sample Size (ZriRecordCnt)

### 0.2.2 Latitude and Longitude of California Cities
### Maps of World (California Latitude and Longitude Map, city level)

### 0.2.3 Foursquare Location Data to Get Nearby Venues
### Merge above tables to get nearby venues with city name, latitude and longitude
### The nearby venues table after data wrangling
### Then, new table will display the top 10 venues for each city.
### Next, KMeans cluster cities by top 10 venues.
### Finally, conclusion will compare cluster labels with SizeRank, Zri, and ZriRecordCnt.

## 1. Get data

### 1.1 Zillow Rent Index (city level)

In [2]:
import pandas as pd # library for data analsysis

In [207]:
df=pd.read_csv('http://files.zillowstatic.com/research/public/City/City_Zri_AllHomesPlusMultifamily_Summary.csv')
df.head()

,Date,RegionName,State,Metro,County,SizeRank,Zri,MoM,QoQ,YoY,ZriRecordCnt
0,2019-12-31,New York,NY,New York-Newark-Jersey City,Queens County,0,2400,0.0028,0.0196,0.0371,2099299
1,2019-12-31,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,1,2840,0.0026,0.0142,0.0320,824116
2,2019-12-31,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,2,1410,0.0123,0.0204,0.0056,898628
3,2019-12-31,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,1710,0.0003,-0.0121,0.0103,807202
4,2019-12-31,San Antonio,TX,San Antonio-New Braunfels,Bexar County,4,1200,0.0068,0.0016,-0.0007,518784


In [520]:
df_CA=df[df['State']=='CA'][['RegionName','SizeRank','Zri','ZriRecordCnt']].reset_index(drop=True)
df_CA.columns=['City','SizeRank','Zri','ZriRecordCnt']
df_CA.head()

,City,SizeRank,Zri,ZriRecordCnt
0,Los Angeles,1,2840,824116
1,San Diego,8,2610,406428
2,San Jose,11,3130,257852
3,San Francisco,14,4220,182102
4,Sacramento,29,1700,185705


### 1.2 Latitude and Longitude of California Cities

In [11]:
import requests # library to handle requests

In [12]:
url='https://www.mapsofworld.com/usa/states/california/lat-long.html'
web_text=requests.get(url).text

In [134]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(web_text,'lxml')
#print(soup.prettify())

In [135]:
result_table=soup.find_all('table',{'class':'tableizer-table'})
#result_table

In [122]:
ths = result_table[1].find_all('th')
headings = [th.text.strip() for th in ths]
headings

['Location', 'Latitude', 'Longitude']

In [130]:
data_list=[]
for i in [1,2]:
    for tr in result_table[i].find_all('tr'):
        tds = tr.find_all('td')
        if tds != []:
            data_list.append([td.text.strip() for td in tds])

In [378]:
df_coords=pd.DataFrame(data_list,columns=headings)
df_coords.columns=['City','Latitude','Longitude']
df_coords.head()

,City,Latitude,Longitude
0,Acalanes Ridge,37.9,-122.08
1,Acampo,38.17,-121.28
2,Acton,34.5,-118.19
3,Adelanto city,34.59,-117.44
4,Adin,41.2,-120.95


In [379]:
import re

In [380]:
r = re.compile(r'.*(city)$')
withcity=df_coords[df_coords['City'].apply(lambda x: bool(r.match(x)))].City
newlist=list(item[0] for item in withcity.str.rsplit(' ',1).tolist())
df_coords.replace(withcity.tolist(),newlist,inplace=True)

In [381]:
r = re.compile(r'.*(town)$')
withtown=df_coords[df_coords['City'].apply(lambda x: bool(r.match(x)))].City
newlist2=list(item[0] for item in withtown.str.rsplit(' ',1).tolist())
df_coords.replace(withtown.tolist(),newlist2,inplace=True)

In [382]:
r = re.compile(r'.*\)$')
withsign=df_coords[df_coords['City'].apply(lambda x: bool(r.match(x)))].City
newlist3=list(item[-1].rstrip(')') for item in withsign.str.rsplit('(',1).tolist())
df_coords.replace(withsign.tolist(),newlist3,inplace=True)

In [383]:
df_coords.iloc[674,0]='La Canada Flintridge'
df_coords.iloc[840,0]='Mc Farland'
df_coords.iloc[1307,0]='Saint Helena'
df_coords.iloc[595,0]='Hilmar'
df_coords.iloc[211,0]='Carmel Valley'
df_coords.iloc[1392,0]='Tuolumne'
df_coords.iloc[617,0]='Idyllwild'
df_coords.iloc[1237,0]='The Sea Ranch'

In [385]:
#df_coords[df_coords.City.duplicated(keep=False)]
df_coords.drop_duplicates(subset ='City', keep = 'last', inplace = True) 

In [472]:
print(df_CA.shape, df_coords.shape)

(605, 3) (1504, 3)


In [521]:
CA_city_zri=df_CA.merge(df_coords, on='City')
CA_city_zri.head()

,City,SizeRank,Zri,ZriRecordCnt,Latitude,Longitude
0,Los Angeles,1,2840,824116,34.11,-118.41
1,San Diego,8,2610,406428,32.82,-117.13
2,San Jose,11,3130,257852,37.3,-121.85
3,San Francisco,14,4220,182102,37.76,-122.69
4,Sacramento,29,1700,185705,38.57,-121.47


In [522]:
print(CA_city_zri.shape)

(592, 6)


In [393]:
len(df_CA[(~df_CA.City.isin(CA_city_zri.City))])

13

In [394]:
CA_city_zri=CA_city_zri.astype({'Latitude': 'float64','Longitude': 'float64'})

In [523]:
CA_city_zri.dtypes

City            object
SizeRank         int64
Zri              int64
ZriRecordCnt     int64
Latitude        object
Longitude       object
dtype: object

## 2. Explore and cluster the cities in CA

!conda install -c conda-forge folium --yes

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

### 2.1 California map

In [399]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [400]:
address = 'California'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of California are 36.7014631, -118.7559974.


In [418]:
# create map of Toronto using latitude and longitude values
map_ca = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, label in zip(CA_city_zri['Latitude'], CA_city_zri['Longitude'], CA_city_zri['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ca)  
    
map_ca

### 2.2 Get nearby venues

In [411]:
# The code was removed by Watson Studio for sharing.

In [420]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [423]:
ca_venues = getNearbyVenues(names=CA_city_zri['City'],
                                   latitudes=CA_city_zri['Latitude'],
                                   longitudes=CA_city_zri['Longitude']
                                  )

In [422]:
print(ca_venues.shape)
ca_venues.head()

(13366, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Los Angeles,34.11,-118.41,Planet Smoothie,34.106260,-118.418041,Juice Bar
1,Los Angeles,34.11,-118.41,Hastain Trail,34.106200,-118.411327,Trail
2,Los Angeles,34.11,-118.41,Franklin Canyon Ranch,34.106318,-118.413352,Park
3,Los Angeles,34.11,-118.41,A Cleaner Pool,34.111485,-118.404465,Home Service
4,Los Angeles,34.11,-118.41,Sai Num Song Fung Khong,34.103432,-118.410871,Vietnamese Restaurant


In [416]:
ca_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acampo,2,2,2,2,2,2
Adelanto,2,2,2,2,2,2
Agoura Hills,70,70,70,70,70,70
Agua Dulce,8,8,8,8,8,8
Alameda,12,12,12,12,12,12
Alamo,9,9,9,9,9,9
Albany,1,1,1,1,1,1
Alhambra,43,43,43,43,43,43
Aliso Viejo,71,71,71,71,71,71


In [424]:
print('There are {} uniques categories.'.format(len(ca_venues['Venue Category'].unique())))

There are 454 uniques categories.


### 2.3 Analyze each city

In [435]:
# one hot encoding
ca_onehot = pd.get_dummies(ca_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ca_onehot['Neighborhood'] = ca_venues['Neighborhood'] 

# move neighborhood column to the first column
ca_onehot=ca_onehot[['Neighborhood']+[c for c in ca_onehot if c not in ['Neighborhood']]]

ca_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Alternative Healer,...,Warehouse Store,Water Park,Waterfall,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Los Angeles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Los Angeles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Los Angeles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Los Angeles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Los Angeles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [438]:
ca_onehot.shape

(13366, 455)

In [439]:
ca_grouped = ca_onehot.groupby('Neighborhood').mean().reset_index()
ca_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Alternative Healer,...,Warehouse Store,Water Park,Waterfall,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Acampo,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.5,0.000000,0.00,0.00
1,Adelanto,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
2,Agoura Hills,0.014286,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
3,Agua Dulce,0.000000,0.00,0.0,0.125,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
4,Alameda,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
5,Alamo,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
6,Albany,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
7,Alhambra,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
8,Aliso Viejo,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00
9,Alpine,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00


In [440]:
ca_grouped.shape

(564, 455)

### 2.4 Create the new dataframe and display the top 10 venues for each city

In [525]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [526]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ca_grouped['Neighborhood']

for ind in np.arange(ca_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ca_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acampo,Winery,Business Service,Yoga Studio,Field,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Factory
1,Adelanto,Cosmetics Shop,Department Store,Yoga Studio,Filipino Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Factory,Fair
2,Agoura Hills,Fast Food Restaurant,Deli / Bodega,Pharmacy,Mexican Restaurant,Bakery,Gym / Fitness Center,Gym,Chinese Restaurant,Shipping Store,Athletics & Sports
3,Agua Dulce,Bakery,Convenience Store,Café,Mexican Restaurant,Pizza Place,Gift Shop,Grocery Store,Airport,Eye Doctor,Factory
4,Alameda,Beach,Bus Station,Liquor Store,Home Service,Trail,Tennis Court,Dog Run,Other Great Outdoors,Gym,Park


### 2.5 Cluster cities

In [527]:
# set number of clusters
kclusters = 3

ca_grouped_clustering = ca_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ca_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [528]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ca_merged = CA_city_zri

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ca_merged = ca_merged.merge(neighborhoods_venues_sorted, left_on='City', right_on='Neighborhood')

ca_merged.head() # check the last columns!

,City,SizeRank,Zri,ZriRecordCnt,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Los Angeles,1,2840,824116,34.11,-118.41,1,Los Angeles,Home Service,Park,Trail,Juice Bar,Vietnamese Restaurant,Yoga Studio,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor
1,San Diego,8,2610,406428,32.82,-117.13,0,San Diego,Café,Sandwich Place,Real Estate Office,Hotel,Diner,Doctor's Office,Cajun / Creole Restaurant,Business Service,Frozen Yogurt Shop,Gym
2,San Jose,11,3130,257852,37.3,-121.85,0,San Jose,Vietnamese Restaurant,Fast Food Restaurant,Mexican Restaurant,Marijuana Dispensary,Asian Restaurant,Food Truck,Breakfast Spot,Storage Facility,Furniture / Home Store,Casino
3,Sacramento,29,1700,185705,38.57,-121.47,0,Sacramento,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,Dive Bar,American Restaurant,Japanese Restaurant,Sandwich Place,Breakfast Spot,Salon / Barbershop
4,Fresno,32,1330,152740,36.78,-119.79,0,Fresno,Mexican Restaurant,Hotel,Chinese Restaurant,Video Store,Liquor Store,Sandwich Place,Discount Store,Breakfast Spot,Shopping Mall,Movie Theater


In [529]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ca_merged['Latitude'], ca_merged['Longitude'], ca_merged['Neighborhood'], ca_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3. Conclusion

## 3.1 When demand is high, the housing rental fee is high

In [530]:
ca_merged.groupby('Cluster Labels').mean()

,SizeRank,Zri,ZriRecordCnt
Cluster Labels,,,
0,2536.021484,2329.003906,16427.994141
1,2643.822222,2583.777778,33324.555556
2,4334.285714,2094.285714,8312.000000


#### Zillow Rent Index (ZRI): A smoothed measure of the typical estimated market rate rent across a given region and housing type.ZRI, which is a dollar-denominated alternative to repeat-rent indices, is the mean of rent estimates that fall into the 40th to 60th percentile range for all homes and apartments in a given region, including those not currently listed for rent. 

#### SizeRank measures the region area (lower value means a big city group). 

#### ZriRecordCnt is the sample size (number of rental houses) for each city

#### Therefore, Label 1 city group has high rental fees with large demand (large city).
#### Labels 2 city group has low rental fees with low demand (small city).
#### Label 0 city group is between Label 1 and Label 2 (rental price and demand), but the city has a big size.

## 3.2  Top 3 venues of 1st Most Common Venue for each city group

In [533]:
ca_merged[ca_merged['Cluster Labels']==1].iloc[:,8].value_counts().nlargest(3)

Park            27
Trail            4
Home Service     2
Name: 1st Most Common Venue, dtype: int64

#### The metropolitan has the relaxed common venue.

In [534]:
ca_merged[ca_merged['Cluster Labels']==0].iloc[:,8].value_counts().nlargest(3)

Mexican Restaurant      74
Fast Food Restaurant    55
Pizza Place             30
Name: 1st Most Common Venue, dtype: int64

#### The common living area provide restaurant

In [535]:
ca_merged[ca_merged['Cluster Labels']==2].iloc[:,8].value_counts().nlargest(3)

Business Service    6
Winery              1
Name: 1st Most Common Venue, dtype: int64

#### The suburb and countryside are with business service.